In [1]:
params = {
    # exp
    "seed": 0,
    
    "data_size": 1000,
    "n_features": 2,

    "class1_mean_min":0, 
    "class1_mean_max":1,
    "class2_mean_min":2, 
    "class2_mean_max":3, 

    "class1_cov_min":1, 
    "class1_cov_max":2,
    "class2_cov_min":1, 
    "class2_cov_max":2.5, 
    
}


In [2]:
import Data.data_provider as dp
import numpy as np

for nf in np.arange(2, 50, 2).tolist():
    # print(">>>", nf)
    X, y, tp = dp.make_classification_gaussian_with_true_prob(params["data_size"], 
                                                            nf, 
                                                            class1_mean_min = params["class1_mean_min"], 
                                                            class1_mean_max = params["class1_mean_max"],
                                                            class2_mean_min = params["class2_mean_min"], 
                                                            class2_mean_max = params["class2_mean_max"], 
                                                            class1_cov_min = params["class1_cov_min"], 
                                                            class1_cov_max = params["class1_cov_max"],
                                                            class2_cov_min = params["class2_cov_min"], 
                                                            class2_cov_max = params["class2_cov_max"], 
                                                            bais_accuracy=0.80)


2: 1.0,
4: 1.0,
6: 1.0,
8: 1.0,
10: 1.0,
12: 1.0,
14: 1.0,
16: 1.0,
18: 1.0,
20: 1.0,
22: 1.0,
24: 1.0,
26: 1.0,
28: 1.0,
30: 1.0,
32: 1.0,
34: 1.0,
36: 1.0,
38: 1.0,
40: 1.0,
42: 1.0,
44: 1.0,
46: 1.0,
48: 1.0,
